In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re

def calculate_2dft(input):
    ft = np.fft.ifftshift(input)
    ft = np.fft.fft2(ft)
    return np.fft.fftshift(ft)

def calculate_2dift(input):
    ift = np.fft.ifftshift(input)
    ift = np.fft.ifft2(ift)
    return np.fft.fftshift(ift).real

def calculate_distance_from_centre(coords, centre):
    return np.sqrt((coords[0] - centre) ** 2 + (coords[1] - centre) ** 2)

def find_symmetric_coordinates(coords, centre):
    return (centre + (centre - coords[0]), centre + (centre - coords[1]))

def calculate_metrics(tp, fp, fn, tn):
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return precision, recall, f1_score

base_path ="/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/computational files/qc/Stitching/Raw_tissue plots"
patient_folders = ["IWP5", "IWP6", "IWP7", "IWP8", "IWP9", "IWP10", "IWP11", "IWR5", "IWR6", "IWR7", "IWR8", "IWR9", "IWR10", "IWR11"]  
max_regions = 10
term = 50
threshold_no = 0.1
threshold_high = 0.5

for patient_folder in patient_folders:
    patient_tp = patient_fp = patient_fn = patient_tn = 0

    print(f"\nPatient: {patient_folder}")
    for region_number in range(1, max_regions + 1):
        region_tp = region_fp = region_fn = region_tn = 0
        
        for folder in ['no', 'high']:
            folder_path = os.path.join(base_path, patient_folder, f"{patient_folder}Reg{region_number:02}", folder)

            if not os.path.exists(folder_path):
                print(f"Folder does not exist: {folder_path}")
                continue
            
            print(f"Processing folder: {folder_path}")

            folder_name = os.path.basename(os.path.dirname(folder_path))
            current_folder_name = os.path.basename(folder_path.rstrip('/'))
            match = re.search(r'IWP\d+Reg\d+', folder_path)
            plot_title_prefix = match.group() if match else folder_name

            all_files = os.listdir(folder_path)
            png_files = [f for f in all_files if f.endswith('.png')]

            for image_filename in png_files:
                full_image_path = os.path.join(folder_path, image_filename)
                image = plt.imread(full_image_path)
                image = image[:, :, :3].mean(axis=2)

                ft = calculate_2dft(image)
                array_size = len(image)
                centre = int((array_size - 1) / 2)
                coords_left_half = ((x, y) for x in range(array_size) for y in range(centre + 1))
                coords_left_half = sorted(coords_left_half, key=lambda x: calculate_distance_from_centre(x, centre))

                rec_image = np.zeros(image.shape)
                individual_grating = np.zeros(image.shape, dtype="complex")

                for idx, coords in enumerate(coords_left_half, start=1):
                    if idx > term:
                        break
                    symm_coords = find_symmetric_coordinates(coords, centre)
                    individual_grating[coords] = ft[coords]
                    individual_grating[symm_coords] = ft[symm_coords]
                    rec_grating = calculate_2dift(individual_grating)
                    rec_image += rec_grating
                    individual_grating[coords] = 0
                    individual_grating[symm_coords] = 0

                #plt.figure()
                #plt.imshow(rec_image, cmap="gray")
                #plt.title(f"{plot_title_prefix} - {current_folder_name} - {image_filename} (Term {term})")
                #plt.axis("off")
                #plt.show()

                avg_intensity = np.mean(rec_image)
                
                if folder == 'no':
                    if avg_intensity > threshold_no:
                        region_fp += 1
                        patient_fp += 1
                    else:
                        region_tn += 1
                        patient_tn += 1
                else:  # folder is 'high'
                    if avg_intensity > threshold_high:
                        region_tp += 1
                        patient_tp += 1
                    else:
                        region_fn += 1
                        patient_fn += 1
        
        precision, recall, f1_score = calculate_metrics(region_tp, region_fp, region_fn, region_tn)
        print(f"\nRegion: {region_number}")
        print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1_score:.2f}")
    
    patient_precision, patient_recall, patient_f1_score = calculate_metrics(patient_tp, patient_fp, patient_fn, patient_tn)
    print(f"\nOverall for Patient {patient_folder}")
    print(f"Precision: {patient_precision:.2f}, Recall: {patient_recall:.2f}, F1 Score: {patient_f1_score:.2f}")
    print("="*30)



Patient: IWP5
Processing folder: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/computational files/qc/Stitching/Raw_tissue plots/IWP5/IWP5Reg01/no
Processing folder: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/computational files/qc/Stitching/Raw_tissue plots/IWP5/IWP5Reg01/high

Region: 1
Precision: 0.69, Recall: 1.00, F1 Score: 0.82
Processing folder: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/computational files/qc/Stitching/Raw_tissue plots/IWP5/IWP5Reg02/no
Processing folder: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/computational files/qc/Stitching/Raw_tissue plots/IWP5/IWP5Reg02/high

Region: 2
Precision: 0.79, Recall: 1.00, F1 Score: 0.88
Processing folder: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/computational files/qc/Stitching/Raw_tissue plots/IWP5/IWP5Reg03/no
Processing folder: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & 